In [1]:
!pip install -q streamlit langchain langchain_experimental langchain-openai langchain-community langchain-text-splitters chromadb python-dotenv pyngrok pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 53.6 MB/s eta 0

In [2]:
# 🔑 ngrok 인증 토큰을 여기에 붙여넣고 실행해주세요!
# "YOUR_AUTHTOKEN_HERE" 부분을 현아의 실제 ngrok 인증 토큰으로 바꿔주세요!
!ngrok config add-authtoken "YOUR_AUTHTOKEN_HERE"
print("✅ ngrok 인증 토큰 등록 완료!")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
✅ ngrok 인증 토큰 등록 완료!


In [4]:
import os

# 직접 환경변수 등록 (가장 확실한 방법)
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# 확인
print("✅ OPENAI_API_KEY 등록 완료:", os.environ.get("OPENAI_API_KEY")[:10] + "...")

✅ OPENAI_API_KEY 등록 완료: YOUR_OPENA...


In [5]:
%%writefile app.py
import os, uuid
import streamlit as st
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage
from langchain.schema.output_parser import StrOutputParser
from langchain_community.document_loaders import TextLoader, PyPDFLoader, WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from io import BytesIO
import tempfile
import asyncio
import requests

# --- Streamlit 애플리케이션 초기 설정 ---
st.set_page_config(page_title="RealView 챗봇 (AI 기반 지역 분석)", layout="wide")
st.title("🏡 RealView 챗봇: AI 기반 지역 분석 챗봇")
st.markdown("AI 기반 지역 시세 및 분위기 분석 챗봇 'RealView'입니다. 궁금한 지역 또는 기타 일반적인 질문에 대해 문의해 주세요! ✨")

# --- 세션 상태 초기화: 애플리케이션 상태 유지를 위함 ---
if "messages" not in st.session_state:
    st.session_state["messages"] = []
if "vectorstore" not in st.session_state:
    st.session_state["vectorstore"] = None
if "web_loaded_urls" not in st.session_state:
    st.session_state["web_loaded_urls"] = []
if "initial_learning_done" not in st.session_state:
    st.session_state["initial_learning_done"] = False

# --- LLM(Large Language Model) 설정 ---
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.7)

# --- 챗봇의 페르소나 및 응답 로직을 정의하는 프롬프트 템플릿 (대화 이력 포함) ---
prompt = ChatPromptTemplate.from_messages([
    ("system",
     """당신은 'RealView'라는 이름의 친절하고 사려 깊은 AI 부동산 비서이자 지식 전문가입니다.
사용자와의 대화를 기억하고, 이전 대화 내용과 현재 질문의 맥락을 연결하여 보다 자연스럽고 유연하게 소통해 주십시오.

사용자가 특정 지역의 부동산 시세, 거래 동향, 생활 분위기, 학군, 편의시설 등 부동산 관련 질문은 물론,
그 외 어떤 일반적인 질문을 하더라도 사용자의 궁금증에 깊이 공감하며, 친근하고 전문적인 어조로 답변해 주십시오.
단순히 답변만 나열하는 것을 넘어, 사용자가 이전에 언급한 내용과 연관 지어 답변하거나,
다음 대화로 자연스럽게 이어질 수 있는 질문이나 관련 정보 제안을 한두 가지 추가하여 대화의 흐름을 주도하십시오.
동일하거나 유사한 정보의 반복을 피하고, 항상 새로운 관점이나 추가적인 통찰력을 제공하도록 노력하십시오.

제공된 '참고 문서' 내용이 사용자의 질문과 가장 관련성이 높으므로, 답변 시 이 문서를 최우선적으로 활용하십시오.
만약 '참고 문서'에 사용자의 질문에 대한 충분한 정보가 없거나, 질문 내용이 '참고 문서'와 전혀 관련이 없다면,
당신의 폭넓은 일반 지식과 상식을 활용하여 최대한 정확하고 유용한 답변을 제공해 주십시오.
단순히 "참고 문서에 없습니다"라고 대답하기보다는, 항상 최선을 다해 정보를 제공하는 것이 중요합니다.

참고 문서: {context}"""),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{q}")
])

chain = prompt | llm | StrOutputParser()

# --- 문서 로딩, 분할(chunking) 및 벡터 저장소 생성을 담당하는 함수 ---
@st.cache_resource
def create_vectorstore(documents):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    chunks = splitter.split_documents(documents)
    emb = OpenAIEmbeddings(model='text-embedding-3-small')
    db = Chroma.from_documents(chunks, emb)
    return db

# URL의 활성화 상태를 확인하는 함수 (캐싱 적용)
@st.cache_data(ttl=3600)
def check_url_active(url):
    try:
        response = requests.head(url, timeout=5, allow_redirects=True)
        return response.status_code == 200
    except requests.exceptions.RequestException:
        return False

# 모든 학습 문서를 저장할 리스트 초기화
all_documents = []

# --- 기본 지식 문서 (realview_data.md) 로딩 ---
os.makedirs("data", exist_ok=True)
realview_doc_path = "data/realview_data.md"

if not os.path.exists(realview_doc_path):
    with open(realview_doc_path, "w", encoding="utf-8") as f:
        f.write(
            "## 강남구 역삼동 지역 개요\n"
            "- 역삼동은 서울 강남구에 위치하며, 테헤란로를 중심으로 상업, 업무 지구가 발달해 있습니다.\n"
            "- 다양한 연령대의 인구가 거주하며, 직주근접을 선호하는 직장인들이 많습니다.\n"
            "- 교육 환경으로는 역삼초등학교, 진선여자중학교/고등학교 등이 있으며, 대치동 학원가와 인접하여 교육열이 높은 편입니다.\n\n"
            "## 강남구 청담동 지역 개요\n"
            "- 청담동은 고급 주거 및 상업 지역으로, 명품 거리와 갤러리 등이 밀집해 있습니다.\n"
            "- 조용하고 프라이빗한 주거 환경을 선호하는 이들에게 인기가 많습니다.\n"
            "- 청담고등학교 등 명문 학군을 갖추고 있으며, 한강변 접근성이 우수합니다.\n"
            "## 서울시 부동산 시장 동향 (2025년 기준 예측)\n"
            "- 2025년 부동산 시장은 기준금리 인하 가능성과 정부의 완화 정책 기조에 따라 점진적인 회복세를 보일 것으로 전망됩니다.\n"
            "- 특히 강남권은 견고한 수요를 바탕으로 상승 여력이 존재하며, 재건축 및 재개발 이슈가 시장에 긍정적인 영향을 미칠 수 있습니다.\n"
            "- 실수요와 투자 수요의 균형이 중요하며, 특정 지역은 여전히 높은 관심을 받을 것으로 예상됩니다.\n\n"
            "## 대한민국 부동산 투자 원칙\n"
            "- 부동산 투자는 장기적인 관점에서 접근해야 합니다.\n"
            "- 입지 분석, 수요-공급 균형, 정책 변화, 금리 동향 등 다양한 요소를 종합적으로 고려하는 것이 중요합니다.\n"
            "- 본인의 자산 상황과 목표에 맞는 투자 전략을 수립하는 것이 성공적인 투자의 핵심입니다.\n\n"
            "## 주요 지역별 특징\n"
            "- 송파구: 잠실을 중심으로 대규모 주거 단지와 상업 시설이 발달. 올림픽공원 등 녹지 공간 풍부.\n"
            "- 용산구: 한강과 남산을 낀 고급 주거 지역. 용산 국제업무지구 개발 계획 등으로 미래 가치 높음.\n"
            "- 성동구: 성수동을 중심으로 젊은 층 유입 증가. 트렌디한 상업 시설과 지식산업센터 활성화."
        )

loader_text = TextLoader(realview_doc_path, encoding='utf-8')
docs_from_text = loader_text.load()
for doc in docs_from_text:
    doc.metadata['source'] = os.path.basename(realview_doc_path)
all_documents.extend(docs_from_text)


# --- 자동 웹 학습 URL 목록 ---
predefined_web_urls = [
    ("네이버 부동산 블로그 - 주택 매매시장 흐름", "https://blog.naver.com/land_admin/223240465227"),
    ("매일경제 부동산 - 서울 아파트 매매 동향 기사", "https://www.mk.co.kr/news/realestate/10862086"),
    ("한국경제 부동산 - 김유미의 똑똑한 부동산 (투자법)", "https://www.hankyung.com/realestate/article/202305228189e"),
    ("렘군 블로그 - 아파트 투자 핵심 원칙", "https://remgun.com/5176"),
    ("KB 부동산 - 주간 주택시장 동향 리포트 (샘플)", "https://kbland.kbstar.com/starlib/upload/KBdata20241029_112702.pdf"),
    ("한국부동산원 - 부동산시장 동향 월간보고서 (2024년 10월)", "https://www.reb.or.kr/reb/statistics/trend/monthly_reports.jsp"),
    ("국토교통부 - 부동산 보도자료 (최신 정책)", "https://www.molit.go.kr/USR/policyTarget/m_24066/dtl.jsp"),
    ("직방 빅데이터 랩 - 2024년 주택시장 전망", "https://news.zigbang.com/zigbanglab/report-4537"),
    ("경제 뉴스 - 한국경제연구원 부동산시장 분석", "https://www.keri.org/research/view.asp?p_no=0&idx=19565&cls_code=&st_str=&s_year=&e_year=&menu=&tab=2&c_seq="),
    ("부동산R114 - 주간 아파트 시황", "https://www.r114.com/news/brief?all=Y")
]

# --- 초기 웹 학습 및 벡터 저장소 구축 로직 (로딩 페이지 표시) ---
if not st.session_state["initial_learning_done"]:
    loading_container = st.container()
    with loading_container:
        st.write("")
        st.write("")
        st.markdown("<h2 style='text-align: center;'>🏡 RealView 챗봇이 지식을 학습 중입니다...</h2>", unsafe_allow_html=True)
        st.markdown("<h4 style='text-align: center;'>잠시만 기다려 주시면 RealView와 대화할 수 있습니다.</h4>", unsafe_allow_html=True)
        st.write("")
        progress_bar = st.progress(0, text="초기 지식 로딩 준비 중...")
        st.write("")
        status_text = st.empty()
        st.write("")

    total_urls = len(predefined_web_urls)

    for idx, (name, url) in enumerate(predefined_web_urls):
        progress_bar.progress((idx + 1) / total_urls, text=f"📚 웹 지식 학습 중: {name} ({(idx + 1)/total_urls:.0%})")
        status_text.caption(f"✨ 현재 학습 중: {name}")

        if check_url_active(url) and url not in st.session_state.web_loaded_urls:
            try:
                if url.lower().endswith('.pdf'):
                    response = requests.get(url)
                    response.raise_for_status()
                    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                        tmp_file.write(response.content)
                        tmp_file_path = tmp_file.name
                    loader_doc = PyPDFLoader(tmp_file_path)
                    docs_from_web = loader_doc.load()
                    os.remove(tmp_file_path)
                else:
                    loader_web = WebBaseLoader(url)
                    docs_from_web = loader_web.load()

                for doc in docs_from_web:
                    doc.metadata['source'] = f"{name} ({url})"
                all_documents.extend(docs_from_web)
                st.session_state.web_loaded_urls.append(url)
            except Exception as e:
                pass

    if all_documents:
        status_text.caption("🧠 최종 지식 구조화 중...")
        st.session_state["vectorstore"] = create_vectorstore(all_documents)
        progress_bar.progress(1.0, text="✅ 모든 지식 학습 완료!")
        status_text.caption("🎉 RealView 챗봇 준비 완료!")
    else:
        st.warning("⚠️ 학습할 문서가 없거나 모든 URL 접근에 실패했습니다. 기본 정보만으로 답변합니다.")

    loading_container.empty()
    st.session_state["initial_learning_done"] = True
    st.rerun()


# --- 챗봇 대화 및 백그라운드 학습/관리 UI (초기 학습 완료 후에만 표시) ---
if st.session_state["initial_learning_done"]:
    st.subheader("RealView 챗봇에게 부동산 및 일반 질문을 해 보세요!")

    # 대화 기록 표시
    for message in st.session_state.messages:
        with st.chat_message(message.type):
            st.markdown(message.content)
            # ✅ 수정된 부분: message.kwargs 존재 여부 확인 로직 강화
            # hasattr(message, 'kwargs')로 먼저 kwargs 속성 존재 여부 확인
            if message.type == "ai" and hasattr(message, 'kwargs') and "sources" in message.kwargs and message.kwargs["sources"]:
                with st.expander("💡 참고 출처 보기"):
                    st.markdown(', '.join(sorted(message.kwargs["sources"])))

    q = st.chat_input("예: 2025년 부동산 시장 전망은? 또는 서울 아파트 투자 시 고려할 점은 무엇인가요?")

    if q:
        st.chat_message("user").markdown(q)
        st.session_state["messages"].append(HumanMessage(content=q))

        with st.spinner("RealView가 열심히 정보를 분석 중..."):
            context_docs = []
            context_string = "제공된 참고 문서가 없어 LLM의 일반 지식으로 답변합니다."
            sources = set()

            if st.session_state["vectorstore"]:
                context_docs = st.session_state["vectorstore"].similarity_search(q, k=5)
                context_string = "\n\n".join(d.page_content for d in context_docs)

                for d in context_docs:
                    if 'source' in d.metadata:
                        sources.add(d.metadata['source'])

            history_for_llm = []
            for msg in st.session_state["messages"][:-1]:
                if isinstance(msg, HumanMessage):
                    history_for_llm.append(HumanMessage(content=msg.content))
                elif isinstance(msg, AIMessage):
                    # ✅ 수정된 부분: getattr(msg, 'kwargs', {})를 사용하여 kwargs가 없는 경우 빈 딕셔너리 전달
                    history_for_llm.append(AIMessage(content=msg.content, kwargs=getattr(msg, 'kwargs', {})))

            raw_answer = chain.invoke({
                'context': context_string,
                'history': history_for_llm,
                'q': q
            })

            relevant_sources = set()
            for doc in context_docs:
                source_url = doc.metadata.get('source')
                if source_url and raw_answer.lower() in doc.page_content.lower():
                    relevant_sources.add(source_url)

            if "참고 문서가 없어" in raw_answer.lower() or "일반 지식으로 답변합니다" in raw_answer.lower():
                relevant_sources.clear()

            if relevant_sources:
                ai_message_object = AIMessage(content=raw_answer, kwargs={"sources": sorted(list(relevant_sources))})
            else:
                ai_message_object = AIMessage(content=raw_answer)

        st.chat_message("assistant").markdown(raw_answer)
        if relevant_sources:
            with st.expander("💡 참고 출처 보기"):
                st.markdown(', '.join(sorted(list(relevant_sources))))
        st.session_state["messages"].append(ai_message_object)


    with st.expander("🛠️ 학습 및 관리 (관리자용)"):
        st.markdown("여기서 PDF 파일을 업로드하거나 새로운 웹 페이지 URL을 학습시켜 RealView 챗봇의 지식을 확장할 수 있습니다.")

        uploaded_files = st.file_uploader(
            "1. 새로운 PDF 파일을 업로드하십시오 (여러 파일 가능)",
            type="pdf",
            accept_multiple_files=True,
            key="pdf_uploader"
        )

        if uploaded_files:
            new_docs_from_pdf = []
            for uploaded_file in uploaded_files:
                with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                    tmp_file.write(uploaded_file.read())
                    tmp_file_path = tmp_file.name

                st.info(f"✨ PDF 파일 '{uploaded_file.name}' 업로드 완료!")
                loader_pdf = PyPDFLoader(tmp_file_path)
                docs_from_pdf = loader_pdf.load()
                for doc in docs_from_pdf:
                    doc.metadata['source'] = uploaded_file.name
                new_docs_from_pdf.extend(docs_from_pdf)
                os.remove(tmp_file_path)

            if new_docs_from_pdf:
                st.session_state["vectorstore"] = create_vectorstore(all_documents + new_docs_from_pdf)
                all_documents.extend(new_docs_from_pdf)
                st.success("✅ 새로운 PDF 문서 학습 및 벡터 저장소 업데이트 완료!")
                st.rerun()


        url_input_manual = st.text_input("2. 새로운 웹 페이지 URL을 입력하십시오 (예: https://example.com/article)", key="manual_url_input")

        if url_input_manual and url_input_manual not in st.session_state.web_loaded_urls:
            if check_url_active(url_input_manual):
                try:
                    with st.spinner(f"새로운 웹 페이지 '{url_input_manual}' 내용을 학습 중... 🌐"):
                        if url_input_manual.lower().endswith('.pdf'):
                            response = requests.get(url_input_manual)
                            response.raise_for_status()
                            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                                tmp_file.write(response.content)
                                tmp_file_path = tmp_file.name
                            loader_doc = PyPDFLoader(tmp_file_path)
                            docs_from_web = loader_doc.load()
                            os.remove(tmp_file_path)
                        else:
                            loader_web = WebBaseLoader(url_input_manual)
                            docs_from_web = loader_web.load()

                        for doc in docs_from_web:
                            doc.metadata['source'] = url_input_manual

                        st.session_state["vectorstore"] = create_vectorstore(all_documents + docs_from_web)
                        all_documents.extend(docs_from_web)
                        st.session_state.web_loaded_urls.append(url_input_manual)
                        st.success(f"🎉 웹 페이지 '{url_input_manual}' 학습 완료 및 벡터 저장소 업데이트 완료!")
                        st.rerun()
                except Exception as e:
                    st.error(f"❌ 웹 페이지 학습 중 오류 발생: {e}. URL을 확인하거나, 특정 기사/블로그 링크를 시도해 보십시오!")
                    st.info("💡 네이버 부동산 뉴스 메인 페이지와 같이 복잡한 구성의 페이지는 내용 추출이 어렵거나 정확하지 않을 수 있습니다. 블로그 글이나 특정 뉴스 기사 URL을 입력하면 더 좋습니다.")
            else:
                st.error(f"❌ 입력된 URL이 활성화되지 않거나 접근할 수 없습니다: {url_input_manual}")
        elif url_input_manual in st.session_state.web_loaded_urls:
            st.info(f"✅ 웹 페이지 '{url_input_manual}'는 이미 학습되었습니다.")

Writing app.py


In [6]:
import os
import time
import socket
from pyngrok import ngrok
from pyngrok.exception import PyngrokNgrokHTTPError, PyngrokNgrokURLError

# 이전 ngrok 터널 종료 (안전하게 시작하기 위해)
try:
    print("기존 ngrok 터널을 종료 시도합니다...")
    ngrok.kill()
    print("기존 ngrok 터널 종료 완료.")
except Exception as e:
    print(f"기존 ngrok 터널 종료 중 오류 발생 (이미 종료되었을 수 있음 또는 실행 중인 ngrok 없음): {e}")

# Streamlit 앱 백그라운드에서 실행
print("Streamlit 앱을 백그라운드에서 실행합니다...")
os.system("streamlit run app.py --server.address 0.0.0.0 --server.port 8501 &>/dev/null &")

# Streamlit 앱이 8501 포트에서 응답할 때까지 기다리는 함수
def wait_for_streamlit_app(port, timeout=90):
    start_time = time.time()
    while True:
        if time.time() - start_time > timeout:
            print(f"❌ {timeout}초 동안 Streamlit 앱이 {port}번 포트에서 시작되지 않았습니다.")
            return False
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.settimeout(1)
                s.connect(("127.0.0.1", port))
            print(f"✅ Streamlit 앱이 {port}번 포트에서 응답을 시작했습니다!")
            return True
        except (socket.error, ConnectionRefusedError):
            print(f"⏳ Streamlit 앱 대기 중... ({int(time.time() - start_time)}초 경과 / 최대 {timeout}초)")
            time.sleep(2)

# Streamlit 앱이 완전히 로드될 때까지 기다립니다.
if not wait_for_streamlit_app(8501, timeout=90):
    print("❌ Streamlit 앱 시작에 실패하여 ngrok 연결을 시도하지 않습니다.")
else:
    # ngrok 터널 열기
    print("ngrok 터널을 연결합니다...")
    try:
        public_url = ngrok.connect(addr="http://127.0.0.1:8501", bind_tls=True)
        print("🌐 Streamlit 앱이 실행되었습니다! 아래 링크를 클릭하세요:")
        print(public_url)
    except (PyngrokNgrokHTTPError, PyngrokNgrokURLError) as e:
        print(f"❌ ngrok 터널 연결 중 오류 발생: {type(e).__name__} - {e}")
        print("pyngrok 또는 ngrok 서비스 상태를 확인해주세요.")
        print("1. 'ngrok config add-authtoken YOUR_AUTH_TOKEN' 명령어로 인증 토큰을 다시 설정해보세요. (반드시 큰따옴표 안에 토큰을 넣어주세요!)")
        print("2. ngrok 유료 플랜이 아닌 경우, 연결 요청이 너무 많아서 일시적으로 차단되었을 수도 있습니다. 잠시 후 다시 시도해 보세요.")
        print("3. 코랩 세션 초기화 후 모든 셀을 순서대로 다시 실행해 보세요.")
    except Exception as e:
        print(f"❌ 예상치 못한 오류 발생: {type(e).__name__} - {e}")

기존 ngrok 터널을 종료 시도합니다...
기존 ngrok 터널 종료 완료.
Streamlit 앱을 백그라운드에서 실행합니다...
⏳ Streamlit 앱 대기 중... (0초 경과 / 최대 90초)
⏳ Streamlit 앱 대기 중... (2초 경과 / 최대 90초)
⏳ Streamlit 앱 대기 중... (4초 경과 / 최대 90초)
✅ Streamlit 앱이 8501번 포트에서 응답을 시작했습니다!
ngrok 터널을 연결합니다...


ERROR:pyngrok.process.ngrok:t=2025-11-12T05:48:00+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: YOUR_AUTHTOKEN_HERE\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"


❌ 예상치 못한 오류 발생: PyngrokNgrokError - The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: YOUR_AUTHTOKEN_HERE\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.
